# MyPy and

![title](buddy.jpg)

# Who am I?

- Philipp Konrad
- Code Monkey at RadarServices
- Trying to be a real monkey when not coding 🐒

You can find the slides 
`https://github.com/gardiac2002/conferences/pydays2019`

(The link is as well on the last slide)

# What are dynamic and static typing?

- **Dynamically typed** -> type checks when code runs (Python)
- **Statically typed** -> at compile time (C, Rust)

**2014: Say hello to PEP 484 (Type Hints)** 

- Python **stays** a dynamically typed language
- But we can create **type hints**
- We can verify type hints before execution (**mypy**).

# Why are type hints awesome?

- Find bugs
- Manage big projects better
- Increase readability

# Let's dive in!

![title](woodie.jpg)

## Type Hints are a feature of Python3

## And you can use `mypy` to check your type hints

In [42]:
!mypy your_amazing_project  # pip install mypy :)

your_amazing_project/test.py:2: error: No return value expected
your_amazing_project/test.py:6: error: Incompatible return value type (got "None", expected "int")


In [43]:
from IPython.core.magic import register_cell_magic
from IPython import get_ipython
from mypy import api

@register_cell_magic
def mypybuddy(line, cell):  # for this presentation only
    cell = '\n' + cell
    mypy_result = api.run(['-c', cell] + line.split())

    if mypy_result[0]:  
        print(mypy_result[0])

    if mypy_result[1]:  
        print(mypy_result[1])

In [44]:
%%mypybuddy

def nothing() -> None:
    return 1

<string>:4: error: No return value expected



## Mypy does not check for side effects or really bad code

In [45]:
%%mypybuddy

def add_one(number: float) -> float:
    print('[Side effect] Lunch nuclear missiles!')
    return number + 1.

## By default, `mypy` does not demand typing.

In [46]:
%%mypybuddy

def add_two(number):
    return number + 2

## But we can be `--strict` about it

In [47]:
%%mypybuddy --strict

def add_three(number):
    return number + 3

<string>:3: error: Function is missing a type annotation



## How does typing work - with annotations!

- `name` ==> string
- `greeting` ==> string 
- and we return `->` `None`

In [48]:
def say_hello(name: str, greeting: str = 'Hello') -> None:
    print(greeting + name)

## How does `mypy` actually get those values, `name: str`?

In [49]:
say_hello.__annotations__

{'name': 'str', 'greeting': 'str', 'return': 'None'}

## **Python3.6+:** Inline type hints for variables

In [50]:
%%mypybuddy

all_the_truth: float = 42.0

## For variables: `typing.get_type_hints` !

In [51]:
import __main__
from typing import get_type_hints

all_the_truth: float = 42.1

get_type_hints(__main__)

{'all_the_truth': float,
 'magicians': typing.List[__main__.Magician],
 'coordinates_pydays': typing.Tuple[float, float],
 'magician': typing.Dict[str, typing.Union[str, int]],
 'hogwarts': typing.List[__main__.Magician]}

## Or  `typing.get_type_hints` for functions

In [52]:
def say_hello(name: str, greeting: str = 'Hello') -> None:
    print(greeting + name)

get_type_hints(say_hello)

{'name': str, 'greeting': str, 'return': NoneType}

## Lists, Tuples, Dictionaries

In [53]:
%%mypybuddy

magicians: list = ['Harry', 'Hermione', 'Albus']
coordinates_pydays: tuple = (48.239481, 16.377372)
magician: dict = {'name': 'Harry', 'age': 18}

## But we want the types to be more precise!

In [54]:
# I am in the standard library - yeeeey!
from typing import Dict, List, Tuple

In [55]:
magicians: List[str] = ['Harry', 'Hermione', 'Albus']

In [56]:
coordinates_pydays: Tuple[float, float] = (48.239481, 16.377372)

In [57]:
magician: Dict[str, str] = {'name': 'Harry', 'age': '18'}

## `mypy --strict` helps us to be precise

In [58]:
%%mypybuddy --strict

magicians: list = ['Harry', 'Hermione', 'Ingrid', 'Albus']

<string>:3: error: Implicit generic "Any". Use "typing.List" and specify generic parameters



## I want my dictionary to contain `str` or `int` as values 

In [59]:
from typing import Dict, Union

magician: Dict[str, Union[str, int]] = {'name': 'Harry', 'age': 18}

## **Aliases** for better readability

In [60]:
Magician = Dict[str, Union[str, int, float]]

In [61]:
%%mypybuddy

from typing import List, Dict, Union

Magician = Dict[str, Union[str, int, float]]
magician: Magician = {'name': 'Harry', 'age': '18'}

## **Aliases** can be used in other type hints

In [62]:
magicians: List[Magician] = [
    {'name': 'Harry', 'age': 18},
    {'name': 'Hermione', 'age': 18},
    {'name': 'Albus', 'age': float('NaN')},
]

## BUT, do not go fully berserk with type aliases!

In the end, `mypy` uses the primitive types 

like `Dict[str, Union[str, int, float]]` etc.

And you do **not** create new types / classes like a real `Magician`!

## If you want a new `type`, better create a class / dataclass

In [63]:
from dataclasses import dataclass

@dataclass
class Magician:
    name: str
    age: int

# By the way, classes create new types :)
type(Magician('Luna', 21))

__main__.Magician

In [64]:
type(Magician)

type

# `typing.Optional` for optional parameters

In [65]:
from typing import Optional
from random import choice

Houses = List[str]
Magicians = List[Magician]
HouseAssignment = Dict[str, str]

def sorting_hat(magicians: Magicians, houses: Optional[Houses]=None) \
    -> HouseAssignment:

    if houses is None:
        houses = ['Gryffindor', 'Hufflepuff', 'Ravenclaw', 'Slytherin']
    
    return {magician['name']: choice(houses) for magician in magicians}

In [66]:
magicians: Magicians = [{'name': 'Harry', 'age': 18}, 
                        {'name': 'Hermione', 'age': 18},]
sorting_hat(magicians)

{'Harry': 'Hufflepuff', 'Hermione': 'Ravenclaw'}

## If it walks like a duck and it quacks like a duck, then it must be a duck

But what if we have a `tuple` of `Magicians`?

In [67]:
from typing import Iterable

# Magicians = List[Magician]
Magicians = Iterable[Magician]

## Returning `None`, `Any` or `NoReturn`

In [68]:
from typing import NoReturn, Any

def take_red_pill() -> NoReturn:
    raise NotImplementedError()

def nothing_will_be_returned() -> None:
    pass

# I accept everything
def printer(anything: Any) -> None:
    print(str(anything))

## A `TypeVar` for every situation

In [69]:
%%mypybuddy

from typing import TypeVar, Sequence
T = TypeVar('T')

def first(seq: Sequence[T]) -> T:   # Generic function
    return seq[0]

the_answer_of_life = first([42, 21, 10, 5])
rector = first(['Albus', 'Severus'])

reveal_locals()  # I reveal the inferred type!

<string>:12: error: Revealed local types are:
<string>:12: error: rector: builtins.str*
<string>:12: error: the_answer_of_life: builtins.int*



## What if we want to return and take a function?

In [70]:
from typing import Callable, Any

def log(func: Callable) -> Callable:
    # just some decorator logic ...
    return func

@log
def example(a: int, b: int) -> int:
    return a + b

In [71]:
%%mypybuddy  --strict

from typing import Callable, Any

def log(func: Callable) -> Callable:
    # just some decorator logic ...
    return func

@log
def example(a: int, b: int) -> int:
    return a + b

<string>:5: error: Missing type parameters for generic type



## Specify the parameters of the `Callable`!

- `Callable[[parameters], return_type]`

In [72]:
from typing import Callable
                                # params     #return value
def supply_threes(func: Callable[[int, int], int]) -> int:
    return func(3, 3)


def add(a: int, b: int) -> int:
    return a + b

supply_threes(add)

6

In [73]:
%%mypybuddy --strict

from typing import Callable
                                # params     #return value
def supply_threes(func: Callable[[int, int], int]) -> int:
    return func(3, 3)


def add(a: int, b: int) -> int:
    return a + b

supply_threes(add)

## What about `*args` and `**kwargs`

In [74]:
def function(*args: int, **kwargs: str) -> None:
    pass

function(1, 2, 3)
function(a='a', b='b', c='c')

## What about the length of something? Why do we speak about `len`?

In [75]:
from typing import Sized

def number_students(school: Sized) -> int:
    return len(school)

hogwarts: List[Magician] = [{'name': 'Harry', 'age': 18}, 
                            {'name': 'Albus', 'age': float('NaN')},]
number_students(hogwarts)

2

No `interfaces`, but an object implements certain methods which form a `protocol`:

        We propose to use the term protocols for types supporting 
        structural subtyping. The reason is that the term iterator 
        protocol, for example, is widely understood in the community, 
        and coming up with a new term for this concept in a statically 
        typed context would just create confusion. (PEP554)

The `typing` module provides already several `Protocols`:

- Sized
- Sequence
- Container
- Iterable 
- Awaitable
- ContextManager.

## Define your own Protocols 🎩

Of course, you can define your own `Protocol` by subclassing `typing_extensions.Protocol` and implementing
the methods without a body.

In [76]:
# PEP 554 is still in work and a draft 
# however, it is already implemented in `mypy` and allows Protocols
from typing_extensions import Protocol

In [77]:
from typing import Iterable
from typing_extensions import Protocol

class SupportsClose(Protocol):
    def close(self) -> None:
        pass

# No SupportsClose base class!
class Resource:   
    def release(self) -> None:
        pass
    
    def close(self) -> None:
       self.release()


def close_all(items: Iterable[SupportsClose]) -> None:
    for item in items:
        item.close()

close_all([Resource(), open('requirements.txt')])  # Okay!

In [78]:
%%mypybuddy --strict

class Magician:
    
    def __init__(self, name: str):
        self.name = name
        self.old_name: str = ''
    
    def transform(self, name: str) -> 'Magician':
        transformed = Magician(name)
        transformed.old_name = self.name
        return transformed

## How do use types / classes which we just defined?

In [79]:
from __future__ import annotations  # Python4!

class Magician:
    
    def __init__(self, name: str):
        self.name = name
        self.old_name: str = ''
    
    def transform(self, name: str) -> Magician:
        transformed = Magician(name)
        transformed.old_name = self.name
        return transformed
    
    # earlier
    def old_transform(self, name: str) -> 'Magician':
        ...

# Thank you for your attention!

![title](doggie.jpg)

`https://github.com/gardiac2002/conferences/pydays`